In [12]:
import random 
import networkx as nx
from itertools import combinations

In [ ]:
def is_complete(solution, terminals):
    temp_graph = nx.Graph()
    temp_graph.add_edges_from(solution)
    # Ensure all terminals are added to the graph
    temp_graph.add_nodes_from(terminals)

    connected_components = nx.node_connected_component(temp_graph, terminals[0])
    return set(terminals).issubset(connected_components)

In [ ]:
def cost(solution, graph):
    return sum(graph[u][v]["weight"] for u, v in solution)

In [ ]:
def possible_edges_to_add(graph, partial_solution, terminals):

    temp_graph = nx.Graph()
    temp_graph.add_edges_from(partial_solution)
    nodes_in_solution = set(temp_graph.nodes)

    nodes_in_solution.update(terminals)

In [ ]:
def calculate_lower_bound(graph, terminals):
    shortest_path_bound = 0

    for i in range(len(terminals)):
        for j in range(i + 1, len(terminals)):
            try:
                # Compute the shortest path length between terminals[i] and terminals[j]
                path_length = nx.shortest_path_length(graph, source=terminals[i], target=terminals[j], weight='weight')
                shortest_path_bound = max(shortest_path_bound, path_length)
            except nx.NetworkXNoPath:
                shortest_path_bound = float('inf')
                break
    return shortest_path_bound

In [ ]:
def branch_and_bound(graph, terminals):
    best_solution = None
    best_cost = float("inf")

    mst = nx.minimum_spanning_tree(graph)
    best_solution = set(mst.edges)
    best_cost = mst.size(weight="weight")

    def BnB(partial_solution, lower_bound):
        nonlocal best_solution, best_cost

        if is_complete(partial_solution, terminals):
            current_cost = cost(partial_solution, graph)
            if current_cost < best_cost:
                best_solution = partial_solution.copy()
                best_cost = current_cost
            return

        # Prune branches that exceed the best cost
        if lower_bound >= best_cost:
            return

        # Branch on possible edges
        for edge in possible_edges_to_add(graph, partial_solution, terminals):
            new_partial_solution = partial_solution.copy()
            new_partial_solution.add(edge)
            new_lower_bound = calculate_lower_bound(graph, terminals) 
            BnB(new_partial_solution, new_lower_bound)

    # Start the recursion with an empty solution
    initial_solution = set()
    initial_lower_bound = calculate_lower_bound(graph, terminals) 
    BnB(initial_solution, initial_lower_bound)

    return best_solution, best_cost

In [ ]:
def test_branch_and_bound():
   
    # Create the graph of the worst instance of error ratio example
    G = nx.Graph()

    G.add_edge('A', 'T1', weight=5)
    G.add_edge('A', 'B', weight=7)
    G.add_edge('A', 'D', weight=9)
    G.add_edge('T1', 'D', weight=8)
    G.add_edge('A', 'T2', weight=5)
    G.add_edge('A', 'C', weight=3)
    G.add_edge('A', 'T3', weight=5)
    G.add_edge('T1', 'B', weight=7)
    G.add_edge('B', 'T2', weight=3)
    G.add_edge('T2', 'C', weight=6)
    G.add_edge('C', 'T3', weight=4)
    G.add_edge('T3', 'D', weight=7)

    terminals = ['T1', 'T2', 'T3']

    best_solution, best_cost = branch_and_bound(G, terminals)
    print("\nBest Steiner tree solution (edges):", best_solution)
    print("Total cost of the Steiner tree:", best_cost)

In [98]:
test_branch_and_bound()


Best Steiner tree solution (edges): {('A', 'T1'), ('A', 'T2'), ('A', 'T3')}
Total cost of the Steiner tree: 15
